### Human in The Loop

In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [3]:
from uuid import uuid4

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [4]:
tool = TavilySearchResults(max_results=2)

In [5]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
            )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [15]:
messages = [HumanMessage(content="Whats the weather in Ankara?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41eb0bee-54c5-478f-9f41-089a59a27b8e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read'

In [18]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='8c8065b7-1a54-4ec4-9850-b1067e2553e6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41eb0bee-54c5-478f-9f41-089a59a27b8e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkT

In [19]:
abot.graph.get_state(thread).next

('action',)

### continue after interrupt

In [20]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Ankara\', \'region\': \'Ankara\', \'country\': \'Turkey\', \'lat\': 39.9272, \'lon\': 32.8644, \'tz_id\': \'Europe/Istanbul\', \'localtime_epoch\': 1736972982, \'localtime\': \'2025-01-15 23:29\'}, \'current\': {\'last_updated_epoch\': 1736972100, \'last_updated\': \'2025-01-15 23:15\', \'temp_c\': 4.2, \'temp_f\': 39.6, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.0, \'wind_kph\': 6.5, \'wind_degree\': 196, \'wind_dir\': \'SSW\', \'pressure_mb\': 1020.0, \'pressure_in\': 30.12, \'precip_mm\': 0.01, \'precip_in\': 0.0, \'humidity\': 100, \'cloud\': 0, \'feelslike_c\': 2.6, \'feelslike_f\': 36.8, \'win

In [21]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='8c8065b7-1a54-4ec4-9850-b1067e2553e6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41eb0bee-54c5-478f-9f41-089a59a27b8e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkT

In [22]:
abot.graph.get_state(thread).next

()

In [23]:
messages = [HumanMessage("Who won the 2024 US presidential election?")]
thread = {"configurable": {"thread_id": "44"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TjT29uwWAg3yuGYOi9uHEtKh', 'function': {'arguments': '{"query":"2024 US presidential election winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 156, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-074effd7-6dc5-404b-a5b2-722f9b0d169f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 US presidential election winner'}, 'id': 'call_TjT29uwWAg3yuGYOi9uHEtKh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 156, 'output_tokens': 24, 'total_tokens': 180, 'input_tok

In [31]:
last = abot.graph.get_state(thread)
print(last.values["messages"][-1].content)

Donald Trump won the 2024 US presidential election, defeating Harris to retake the White House. He was elected as the 47th president of the United States in a historic comeback for a second term.


### Modify State